<a href="https://colab.research.google.com/github/MarcStorm/pommerman/blob/master/pommerman/AlexMathias.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
'''Clone and install MultiAgentLearning/playground'''

import os
!cd /content

!git clone https://github.com/MultiAgentLearning/playground
os.chdir('/content/playground')
!pip install -U .

/bin/sh: line 0: cd: /content: No such file or directory
fatal: destination path 'playground' already exists and is not an empty directory.


FileNotFoundError: [Errno 2] No such file or directory: '/content/playground'

In [58]:
# import os
import sys
import numpy as np
import torch
import random
import datetime
from util import flatten_state, flatten_state_no_board
from pommerman.agents import SimpleAgent, RandomAgent, PlayerAgent, BaseAgent
from pommerman.configs import ffa_v0_fast_env
from pommerman.envs.v0 import Pomme
from pommerman.characters import Bomber
from pommerman import utility, characters
from pommerman.constants import Action
from collections import deque
from torch.nn.parameter import Parameter
import torch.nn.init as init
import torch.nn.functional as F
import torch.optim as optim
import torch.nn as nn
import time

############### CUDA SETUP ################

use_cuda = False#torch.cuda.is_available()
device = torch.device("cuda:0" if use_cuda else "cpu")

#if use_cuda
#print('CUDA IS ON: {}'.format(use_cuda))
#if use_cuda:
#    torch.set_default_tensor_type('torch.cuda.FloatTensor')

#def get_cuda(x):
#    """ Converts tensors to cuda, if available. """
#    if use_cuda:
#        return x.cuda()
#    return x

#def get_numpy(x):
#    """ Get numpy array for both cuda and not. """
##    if use_cuda:
#        return x.cpu().data.numpy()
#    return x.data.numpy()

#torch.cuda.is_available()

############### AGENT AND ENVIRONMENT SETUP

class TrainingAgent(BaseAgent):

    def __init__(self, character=characters.Bomber):
        super().__init__(character)


    def act(self, obs, action_space):
        return 0

'''
Agent class that does nothing
'''
class StopAgent(BaseAgent):

    def __init__(self, character=characters.Bomber):
        super(StopAgent,self).__init__(character)
    
    def act(self, obs, action_space):
        return 0
    
    
# Instantiate the environment
config = ffa_v0_fast_env()
env = Pomme(**config["env_kwargs"])

# Add four agents
agents = []
agents = {
    '0' : StopAgent(config["agent"](0, config["game_type"])),
    '1' : StopAgent(config["agent"](1, config["game_type"])),
    '2' : StopAgent(config["agent"](2, config["game_type"])),
    '3' : TrainingAgent(config["agent"](3, config["game_type"]))
}
env.set_agents(list(agents.values()))
env.set_training_agent(3)
env.set_init_game_state(None)


############### NETWORK SETUP ####################

batch_norm=False
in_channels = 3
out_channels = 3
kernel_size = 5

class PolicyNet(nn.Module):

    def __init__(self, n_inputs, n_hidden, n_outputs, learning_rate):
        super(PolicyNet, self).__init__()
        # network
        self.other_shape = [3]

        #Input for conv2d is (batch_size, num_channels, width, height)
        self.conv1 = nn.Conv2d(in_channels = in_channels, out_channels=out_channels,
                               kernel_size=kernel_size, stride=1, padding=2)

        self.conv2 = nn.Conv2d(in_channels = in_channels, out_channels=out_channels,
                               kernel_size=kernel_size, stride=1, padding=2)

        self.conv3 = nn.Conv2d(in_channels = in_channels, out_channels=out_channels,
                               kernel_size=kernel_size, stride=1, padding=2)

        self.convolution_out_size = 11*11*3

        self.ffn_input_size = n_inputs

        self.ffn = nn.Sequential(
            nn.Linear(n_inputs, n_hidden),
            nn.ReLU(),
            nn.Dropout(0.25),
            nn.Linear(n_hidden, n_hidden),
            nn.ReLU(),
            nn.Dropout(0.25),
            nn.Linear(n_hidden, n_hidden),
            nn.ReLU(),
            nn.Linear(n_hidden, n_hidden),
            nn.ReLU(),
            nn.Dropout(0.25),
            nn.Linear(n_hidden, n_outputs),
        )

        self.activation = F.relu

        if batch_norm:
            self.bn1 = nn.BatchNorm2d(11)
        else:
            self.bn1 = lambda x: x
            self.bn2 = lambda x: x
            self.bn3 = lambda x: x

        self.ffn.apply(self.init_weights)

        self.optimizer = optim.Adam(self.parameters(), lr=learning_rate)

    def forward(self, x):
        board_length = len(x[0]['board'])
        completeBoard = [[
                        [[state['board'][x,y] for y in range(board_length)] for x in range(board_length)],
                        [[state['bomb_blast_strength'][x,y] for y in range(board_length)] for x in range(board_length)],
                        [[state['bomb_life'][x,y] for y in range(board_length)] for x in range(board_length)]
                        ] for state in x]

        completeBoard = np.asarray(completeBoard)
        completeBoard = torch.tensor(completeBoard,device=device)
        completeBoard = completeBoard.float()
        boardVariable = torch.autograd.Variable(completeBoard)
        board = self.conv1(boardVariable)
        board = self.bn1(board)
        board = self.activation(board)
        board = self.conv2(board)
        board = self.bn1(board)
        board = self.activation(board)
        board = self.conv3(board)
        board = self.bn1(board)
        board = self.activation(board)
        x2 = board.view(-1, self.convolution_out_size)
        x = flatten_state_no_board(x).to(device)
        x = torch.cat([x2, x], dim=1)
        x = self.ffn(x)
        return F.softmax(x, dim=1)

    def loss(self, action_probabilities, returns):
        return -torch.mean(torch.mul(torch.log(action_probabilities), returns))

    def init_weights(m, *args):
        if type(m) == nn.Linear:
            torch.nn.init.xavier_uniform(m.weight)
            m.bias.data.fill_(0.01)

    # Function from old network
    def update_params(self, new_params, tau):
        params = self.state_dict()
        for k in params.keys():
            params[k] = (1-tau) * params[k] + tau * new_params[k]
        self.load_state_dict(params)

def compute_returns(rewards, discount_factor):
    """Compute discounted returns."""
    returns = np.zeros(len(rewards))
    returns[-1] = rewards[-1]
    for t in reversed(range(len(rewards)-1)):
        returns[t] = rewards[t] + discount_factor * returns[t+1]
    return returns

############### INITIALIZE NETWORK AND TRAINING ##################

n_inputs = 372
n_hidden = 500
n_outputs = env.action_space.n

num_episodes = 10
discount_factor = 0.9
learning_rate = 0.001
val_freq = 25

# setup policy network

policy = PolicyNet(n_inputs, n_hidden, n_outputs, learning_rate)
policy.to(device)

# train policy network

try:
    training_rewards, losses = [], []
    print('start training')
    for i in range(num_episodes):
        print("episode",i+1)
        rollout = []
        s = env.reset()
        done = False
        #policy.train()
        times = []
        while not done:
            startTime = time.perf_counter()
            # generate rollout by iteratively evaluating the current policy on the environment
            with torch.no_grad():
                a_prob = policy(np.atleast_1d(s[0]))
                #a_prob = policy(s[0])
                #print(s[0])
            a = (np.cumsum(a_prob.cpu().numpy()) > np.random.rand()).argmax() # sample action
            
            actions = env.act(s)
            actions.insert(0,a)
            
            #print(actions)
            
            s1, r, done, _ = env.step(actions)
            #print(r)
            rollout.append((s[0], int(a), r[0]))
            #print("\n\nrollout:",rollout,"\n\n")
            s = s1
            times.append(time.perf_counter()-startTime)
        # prepare batch
        print("Steps:",len(times),"Total time:",sum(times),"Mean time:",np.mean(times))
        if(i % 10 == 0):
            
            print('done with episode:',i)
        rollout = np.array(rollout)
        states = np.vstack(rollout[:,0])
        print(rollout[:,1])
        print(type(rollout))
        print(type(rollout[:,1]))
        actions = torch.tensor(rollout[:,1])
        #print(actions)
        rewards = np.array(rollout[:,2], dtype=float)
        returns = compute_returns(rewards, discount_factor)
        # policy gradient update
        policy.optimizer.zero_grad()
        a_probs = policy([s[0] for s in states]).cpu()
        print(a_probs)
        print()
        print(torch.from_numpy(actions).unsqueeze(-1))
        a_probs = a_probs.gather(1, actions.unsqueeze(-1)).view(-1)
        loss = policy.loss(a_probs, torch.from_numpy(returns).float())
        loss.backward()
        policy.optimizer.step()
        # bookkeeping
        training_rewards.append(sum(rewards))
        losses.append(loss.item())
        #policy.eval()
        # print
        if (i+1) % val_freq == 0:
            # validation
            validation_rewards = []
            for _ in range(10):
                s = env.reset()
                reward = 0
                done = False
                while not done:
                    #env.render()
                    with torch.no_grad():
                        probs = policy(np.atleast_1d(s[0]))
                        #a_prob = policy(s[0])
                        a = probs.argmax().item()
                        #print(probs, "max actions: ", a,probs.argmax())
                        
                    actions = env.act(s)
                    actions.insert(0,a)
                    
                    s, r, done, _ = env.step(actions)
                    reward += r[0]
                validation_rewards.append(reward)
                #env.render(close=True)
            print('{:4d}. mean training reward: {:6.2f}, mean validation reward: {:6.2f}, mean loss: {:7.4f}'.format(i+1, np.mean(training_rewards[-val_freq:]), np.mean(validation_rewards), np.mean(losses[-val_freq:])))
    env.close()
    print('done')
except KeyboardInterrupt:
    print('interrupt')    

# plot results
def moving_average(a, n=10) :
    ret = np.cumsum(a, dtype=float)
    ret[n:] = ret[n:] - ret[:-n]
    return ret / n
'''
plt.figure(figsize=(16, 9))
plt.subplot(411)
plt.title('training rewards')
plt.plot(range(1, num_episodes+1), rewards)
plt.plot(moving_average(rewards))
plt.xlim([0, num_episodes])
plt.subplot(412)
plt.title('training lengths')
plt.plot(range(1, num_episodes+1), lengths)
plt.plot(range(1, num_episodes+1), moving_average(lengths))
plt.xlim([0, num_episodes])
plt.subplot(413)
plt.title('training loss')
plt.plot(range(1, num_episodes+1), losses)
plt.plot(range(1, num_episodes+1), moving_average(losses))
plt.xlim([0, num_episodes])
plt.subplot(414)
plt.title('epsilon')
plt.plot(range(1, num_episodes+1), epsilons)
plt.xlim([0, num_episodes])
plt.tight_layout(); plt.show()
'''
## Save file
#t = datetime.date.today().strftime("%Y-%m-%d")
#PATH = "resources/q_agent_{}.pt".format(t)
#torch.save(policy_dqn.state_dict(), PATH)


start training
episode 1
Steps: 801 Total time: 2.1060204196019185 Mean time: 0.00262923897578267
done with episode: 0
[5 2 1 5 3 5 5 4 5 3 2 1 0 2 1 0 0 1 2 2 2 5 0 0 2 0 1 2 2 0 5 0 1 1 1 3 4
 2 3 5 5 0 5 3 2 2 1 4 5 1 1 1 4 2 0 5 1 4 3 4 3 0 1 2 1 3 2 0 2 5 1 4 2 2
 5 2 4 0 2 4 0 2 5 5 0 5 4 5 2 0 0 1 0 2 2 5 2 0 5 2 5 1 0 4 5 1 0 2 2 1 1
 5 2 0 3 2 0 3 5 3 5 4 3 3 1 3 0 0 1 2 3 4 2 1 4 1 5 0 3 1 4 5 5 1 4 0 1 1
 2 0 2 0 3 3 3 0 2 0 0 4 1 1 2 3 2 4 0 4 4 4 3 4 3 1 2 0 4 0 2 5 1 5 1 2 0
 5 0 1 5 4 4 0 0 2 1 4 4 2 5 4 5 1 4 2 4 2 2 2 5 0 2 1 0 3 2 1 5 0 3 4 1 0
 4 0 4 0 1 4 0 4 4 0 5 2 4 3 1 3 2 0 0 1 5 0 1 1 3 3 4 1 0 5 0 4 4 0 0 0 5
 0 0 1 4 4 0 5 1 1 3 4 2 0 4 5 5 4 0 4 5 2 2 2 1 0 1 1 2 1 0 0 0 5 1 1 0 5
 1 5 3 1 5 3 2 0 0 4 0 5 2 1 0 2 5 1 1 4 2 5 5 1 4 5 2 0 4 5 2 0 4 3 3 1 2
 2 1 1 2 1 3 3 1 4 4 4 0 2 0 3 4 0 0 5 3 1 5 2 4 4 3 5 0 4 4 3 1 4 5 0 1 5
 0 0 0 1 0 5 1 0 4 0 1 4 0 0 1 1 4 1 4 1 2 3 5 5 0 1 3 5 2 3 2 5 5 2 5 2 4
 1 4 3 2 2 2 3 4 3 5 4 5 5 1 0 0 5 0 1 0 0 5 0 5 3 0 3 0

TypeError: can't convert np.ndarray of type numpy.object_. The only supported types are: double, float, float16, int64, int32, and uint8.

## Notebook 6.3 Network and Train

In [9]:
   
def compute_returns(rewards, discount_factor):
    """Compute discounted returns."""
    returns = np.zeros(len(rewards))
    returns[-1] = rewards[-1]
    for t in reversed(range(len(rewards)-1)):
        returns[t] = rewards[t] + discount_factor * returns[t+1]
    return returns


num_episodes = 1
discount_factor = 0.9 # reward discount factor (gamma), 1.0 = no discount
learning_rate = 0.001 # you know this by now
val_freq = 25 # validation frequency

# setup policy network

policy = PolicyNet(n_inputs, n_hidden, n_outputs, learning_rate)

# train policy network

try:
    training_rewards, losses = [], []
    print('start training')
    for i in range(num_episodes):
        rollout = []
        s = env.reset()
        done = False
        #policy.train()
        while not done:
            # generate rollout by iteratively evaluating the current policy on the environment
            with torch.no_grad():
                a_prob = policy(np.atleast_1d(s[0]))
                #a_prob = policy(s[0])
                #print(s[0])
            a = (np.cumsum(a_prob.numpy()) > np.random.rand()).argmax() # sample action
            
            actions = env.act(s)
            actions.insert(0,a)
            
            #print(actions)
            
            s1, r, done, _ = env.step(actions)
            #print(r)
            rollout.append((s[0], a, r[0]))
            #print("\n\nrollout:",rollout,"\n\n")
            s = s1
        # prepare batch
        if(i % 10 == 0):
            print('done with episode:',i)
        rollout = np.array(rollout)
        states = np.vstack(rollout[:,0])
        actions = np.vstack(rollout[:,1])
        rewards = np.array(rollout[:,2], dtype=float)
        returns = compute_returns(rewards, discount_factor)
        # policy gradient update
        policy.optimizer.zero_grad()
        a_probs = policy([s[0] for s in states]).gather(1, torch.from_numpy(actions)).view(-1)
        loss = policy.loss(a_probs, torch.from_numpy(returns).float())
        loss.backward()
        policy.optimizer.step()
        # bookkeeping
        training_rewards.append(sum(rewards))
        losses.append(loss.item())
        #policy.eval()
        # print
        if (i+1) % val_freq == 0:
            # validation
            validation_rewards = []
            for _ in range(10):
                s = env.reset()
                reward = 0
                done = False
                while not done:
                    #env.render()
                    with torch.no_grad():
                        probs = policy(np.atleast_1d(s[0]))
                        #a_prob = policy(s[0])
                        a = probs.argmax().item()
                        #print(probs, "max actions: ", a,probs.argmax())
                        
                    actions = env.act(s)
                    actions.insert(0,a)
                    
                    s, r, done, _ = env.step(actions)
                    reward += r[0]
                validation_rewards.append(reward)
                #env.render(close=True)
            print('{:4d}. mean training reward: {:6.2f}, mean validation reward: {:6.2f}, mean loss: {:7.4f}'.format(i+1, np.mean(training_rewards[-val_freq:]), np.mean(validation_rewards), np.mean(losses[-val_freq:])))
    env.close()
    print('done')
except KeyboardInterrupt:
    print('interrupt')    

start training
torch.Size([1, 372])
torch.Size([1, 372])
torch.Size([1, 372])
torch.Size([1, 372])
torch.Size([1, 372])
torch.Size([1, 372])
torch.Size([1, 372])
torch.Size([1, 372])
torch.Size([1, 372])
torch.Size([1, 372])
torch.Size([1, 372])
torch.Size([1, 372])
torch.Size([1, 372])
torch.Size([1, 372])
torch.Size([1, 372])
torch.Size([1, 372])
torch.Size([1, 372])
torch.Size([1, 372])
torch.Size([1, 372])
torch.Size([1, 372])
torch.Size([1, 372])
torch.Size([1, 372])
torch.Size([1, 372])
torch.Size([1, 372])
done with episode: 0
torch.Size([24, 372])
done


### plot results
import matplotlib.pyplot as plt

def moving_average(a, n=10) :
    ret = np.cumsum(a, dtype=float)
    ret[n:] = ret[n:] - ret[:-n]
    return ret / n

plt.figure(figsize=(16,6))
plt.subplot(211)
plt.plot(range(1, len(training_rewards)+1), training_rewards, label='training reward')
plt.plot(moving_average(training_rewards))
plt.xlabel('episode'); plt.ylabel('reward')
plt.xlim((0, len(training_rewards)))
plt.legend(loc=4); plt.grid()
plt.subplot(212)
plt.plot(range(1, len(losses)+1), losses, label='loss')
plt.plot(moving_average(losses))
plt.xlabel('episode'); plt.ylabel('loss')
plt.xlim((0, len(losses)))
plt.legend(loc=4); plt.grid()
plt.tight_layout(); plt.show()

In [83]:
import numpy as np
#completeBoard = [
#    [[[board[x,y]] for y in range(len(board))] for x in range(len(board))]
#    [[[blast_strength[x,y]] for y in range(len(board))] for x in range(len(board))]
#    [[[bomb_life[x,y]] for y in range(len(board))] for x in range(len(board))]
#    ]



abcd = [[[1 for y in range(5)] for x in range(8)],[[1 for y in range(5)] for x in range(8)]]
abcd2 = np.asarray(abcd)

ask = [[4],[4],[4],[3],[2]]

print([i for i in ask])
#as31 = board.reshape([11,11,1])
print(abcd2.shape)
#print(as31)

[[4], [4], [4], [3], [2]]
(2, 8, 5)


In [ ]:
 Example:
# def forward(self, x):
#     x = flatten_state(x)
# where x is np.atleast1d(S[0])

# Flattens a state s on the form list<dict> where each dict contains information of a state
#def flatten_state(s):
    #print("\n Flatten: ",s)
#    return torch.from_numpy(np.array([flatten_state_aux(x) for x in s])).float()

#def flatten_state_aux(s):
    # Lists
    #print("\n Augs: ",s)
#    alive = [1 if x in s['alive'] else 0 for x in range(10,14)]
    #print(alive)
#    board = s['board']
#    bomb_blast_strength = s['bomb_blast_strength']
#    bomb_life = s['bomb_life']
    # Tuples
#    position = s['position']
    # Ints
#    blast_strength = s['blast_strength']
#    can_kick = s['can_kick']
#    ammo = s['ammo']
    # Enums
#    teammate = s['teammate'] #9 for FFA
#    enemies = s['enemies'] #11,12,13 for FFA and training agent id = 0
    
#    a = np.append(np.array(alive),np.array(board).flatten())
#    a = np.append(a,np.array(bomb_blast_strength).flatten())
#    a = np.append(a,np.array(bomb_life).flatten())
#    a = np.append(a,position[0])
#    a = np.append(a,position[1])
#    a = np.append(a,blast_strength)
#    a = np.append(a,can_kick)
#    a = np.append(a,ammo)
    # Commented out as we get size 376 but expected 372. I assume we calculated wrong.
    # Makes sense to ignore these imo
    #a = np.append(a,teammate.value)
    #a = np.append(a,[e.value for e in enemies])
    #print(np.shape(a))
#    return a.astype(float)